In [1]:
import os
import re
import sys
import json
import math
import time
import string
import requests
import io
import numpy as np
import collections
import random
import pickle
import string
import matplotlib.pyplot as plt
import tensorflow as tf
from zipfile import ZipFile
from collections import Counter

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [15]:
# 今回はmodelsフォルダのうち翻訳トレーニング用のseq2seqモデルを使用する
local_repository = 'temp'

if tf.__version__[0]<'1':
    from tensorflow.models.rnn.translate import seq2seq_model
else:
    if not os.path.exists(local_repository):
        from git import Repo
        tf_model_repository = 'https://github.com/tensorflow/models'
        Repo.clone_from(tf_model_repository, local_repository)
        
        # モデルをダウンロードしたあとは、必要な関数をインポート
        sys.path.insert(0, 'temp/tutorials/rnn/translate/')
        import seq2seq_model as seq2seq_model
        import data_utils as data_utils
        
sess = tf.Session()

In [3]:
# パラメーターの設定
learning_rate = 0.1#学習率
lr_decay_rate = 0.99 # 減衰率
lr_decay_every = 100 # 減衰させる頻度
max_gradient = 5.0 # 最大勾配
batch_size = 50
num_layers = 3 # RNNの層の数
rnn_size = 500 # RNNモデルのサイズ
layer_size = 512
generations = 10000
vocab_size = 10000
save_every = 1000
eval_every = 500
output_every = 10
punct = string.punctuation # 句読点を削除

# データのダウンロードと格納
data_dir = 'temp'
data_file = 'eng_ger.txt'
model_path = 'seq2seq_model'
full_model_dir = os.path.join(data_dir, model_path)

In [4]:
# テスト用の英文を３つ設定
test_english = ['hello where is my computer',
                       'the quick brown fox jumped over the lazy dog',
                       'is it going to rain tomorrow']

In [5]:
# データのダウンロード
# モデルディレクトリを作成
if not os.path.exists(full_model_dir):
    os.makedirs(full_model_dir)

# データディレクトリを作成
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    
print('Loading English-German Data')
# データが存在しない場合、ダウンロードして保存
if not os.path.isfile(os.path.join(data_dir, data_file)):
    print('Data not found, downloading Eng-Ger sentences from www.manythings.org')
    sentence_url = 'http://www.manythings.org/anki/deu-eng.zip'
    r = requests.get(sentence_url)
    z = ZipFile(io.BytesIO(r.content))
    file = z.read('deu.txt')
    # データの書式設定
    eng_ger_data = file.decode()
    eng_ger_data = eng_ger_data.encode('ascii', errors='ignore')
    eng_ger_data = eng_ger_data.decode().split('\n')
    # 書き出し
    with open(os.path.join(data_dir, data_file), 'w') as out_conn:
        for sentence in eng_ger_data:
            out_conn.write(sentence + '\n')
else:
    eng_ger_data = []
    with open(os.path.join(data_dir, data_file), 'r') as in_conn:
        for row in in_conn:
            eng_ger_data.append(row[:-1])
print('Done!')

Loading English-German Data
Data not found, downloading Eng-Ger sentences from www.manythings.org
Done!


In [6]:
# データの処理
# 句読点を削除
eng_ger_data = [''.join(char for char in sent if char not in punct) for sent in eng_ger_data]

# 各文をタブで分割     
eng_ger_data = [x.split('\t') for x in eng_ger_data if len(x) >= 1]
[english_sentence, german_sentence] = [list(x) for x in zip(*eng_ger_data)]
english_sentence = [x.lower().split() for x in english_sentence]
german_sentence = [x.lower().split() for x in german_sentence]

In [7]:
# 英語、ドイツ語それぞれ最も出現頻度の高い10000個ずつ辞書作成（残りはラベル０をつける）
# 英語の辞書を処理
all_english_words = [word for sentence in english_sentence for word in sentence]
all_english_counts = Counter(all_english_words)

# -1は0(unknown)の分を引くため
eng_word_keys = [x[0] for x in all_english_counts.most_common(vocab_size-3)]  # -3 because UNK, S, /S is also in there
eng_vocab2ix = dict(zip(eng_word_keys, range(1, vocab_size)))
eng_ix2vocab = {val: key for key, val in eng_vocab2ix.items()}
english_processed = []
for sent in english_sentence:
    temp_sentence = []
    for word in sent:
        try:
            temp_sentence.append(eng_vocab2ix[word])
        except KeyError:
            temp_sentence.append(0)
    english_processed.append(temp_sentence)


# ドイツ語の辞書を処理
all_german_words = [word for sentence in german_sentence for word in sentence]
all_german_counts = Counter(all_german_words)
ger_word_keys = [x[0] for x in all_german_counts.most_common(vocab_size-3)]  # -3 because UNK, S, /S is also in there
ger_vocab2ix = dict(zip(ger_word_keys, range(1, vocab_size)))
ger_ix2vocab = {val: key for key, val in ger_vocab2ix.items()}
german_processed = []
for sent in german_sentence:
    temp_sentence = []
    for word in sent:
        try:
            temp_sentence.append(ger_vocab2ix[word])
        except KeyError:
            temp_sentence.append(0)
    german_processed.append(temp_sentence)

In [8]:
# テスト用の文を、処理
test_data = []
for sentence in test_english:
    temp_sentence = []
    for word in sentence.split(" "):
        try:
            temp_sentence.append(eng_vocab2ix[word])
        except:
            # この単語が辞書に含まれていない場合は0を使用
            temp_sentence.append(0)
    test_data.append(temp_sentence)

In [9]:
# 異なる長さの文ごとに別のモデルを作成
# シーケンスの長さに基づいてバケットを定義し、データを対応するバケットに分割する
x_maxs = [5, 7, 11, 50]
y_maxs = [10, 12, 17, 60]
buckets = [x for x in zip(x_maxs, y_maxs)]
bucketed_data = [[] for _ in range(len(x_maxs))]
for eng, ger in zip(english_processed, german_processed):
    for ix, (x_max, y_max) in enumerate(zip(x_maxs, y_maxs)):
        if (len(eng) <= x_max) and (len(ger) <= y_max):
            bucketed_data[ix].append([eng, ger])
            break

In [18]:
# Seq2Seqとパラメータでモデルを作成するための関数
def translation_model(sess, input_vocab_size, output_vocab_size, buckets, rnn_size, num_layers, max_gradient,
                     learning_rate, lr_decay_rate, forward_only):
    
    model = seq2seq_model.Seq2SeqModel(input_vocab_size, output_vocab_size, buckets, rnn_size, num_layers, max_gradient,
                                      batch_size, learning_rate, lr_decay_rate, forward_only=forward_only, dtypw=tf.float32)
    return(model)

In [19]:
# トレーニング可能なモデルを作成
input_vocab_size = vocab_size
output_vocab_size = vocab_size

translate_model = translation_model(sess=sess, 
                                     input_vocab_size=vocab_size,
                                     output_vocab_size=vocab_size,
                                     buckets=buckets,
                                     rnn_size=rnn_size,
                                     num_layers=num_layers,
                                     max_gradient=max_gradient,
                                     learning_rate=learning_rate,
                                     lr_decay_rate=lr_decay_rate,
                                     forward_only=False)

# テストモデルで同じ変数を再利用
with tf.variable_score(tf.get_variable_scope(), reuse=True):
    test_model = translation_model(sess=sess, 
                                     input_vocab_size=vocab_size,
                                     output_vocab_size=vocab_size,
                                     buckets=buckets,
                                     rnn_size=rnn_size,
                                     num_layers=num_layers,
                                     max_gradient=max_gradient,
                                     learning_rate=learning_rate,
                                     lr_decay_rate=lr_decay_rate,
                                     forward_only=True)
    test_model,batch_size = 1

NameError: name 'seq2seq_model' is not defined